<a href="https://colab.research.google.com/github/SESCDS/sescds/blob/master/Lesson8_PUBG_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUBG Finish Placement Prediction (Kernels Only)
https://www.kaggle.com/c/pubg-finish-placement-prediction

![alt text](https://www.gamespark.jp/imgs/p/PseAHU_gq3GEwRLM5htwMwoLvAaDYAQDAgEA/258064.jpg)

## Скачайте kaggle.json и загрузите его в коллаб запустив ячейки ниже.
![alt text](https://github.com/SESCDS/sescds/blob/master/Lesson7/data/img3.png?raw=true)

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c pubg-finish-placement-prediction

 53% 9.00M/16.9M [00:00<00:00, 34.4MB/s]
100% 16.9M/16.9M [00:00<00:00, 57.7MB/s]
 89% 87.0M/98.3M [00:00<00:00, 70.6MB/s]
100% 98.3M/98.3M [00:00<00:00, 110MB/s] 
 94% 224M/239M [00:02<00:00, 143MB/s]
100% 239M/239M [00:02<00:00, 112MB/s]


In [0]:
import pandas as pd
pd.options.display.max_columns = 999

Если всё выше вы сделали правильно, то успешно считаете датасет и увидете первые его строки командой **head()**.

In [6]:
%%time
train = pd.read_csv('train_V2.csv.zip', compression='zip')

CPU times: user 21.8 s, sys: 2.59 s, total: 24.4 s
Wall time: 24.4 s


In [0]:
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


Описание колонок:

- **groupId** - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
- **matchId** - Integer ID to identify match. There are no matches that are in both the training and testing set.
- **assists** - Number of enemy players this player damaged that were killed by teammates.
- **boosts** - Number of boost items used.
- **damageDealt** - Total damage dealt. Note: Self inflicted damage is subtracted.
- **DBNOs** - Number of enemy players knocked.
- **headshotKills** - Number of enemy players killed with headshots.
- **heals** - Number of healing items used.
- **killPlace** - Ranking in match of number of enemy players killed.
- **killPoints** - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)
- **kills** - Number of enemy players killed.
- **killStreaks** - Max number of enemy players killed in a short amount of time.
- **longestKill** - Longest distance between player and player killed at time of death. This may be misleading, as downing a - player and driving away may lead to a large longestKill stat.
- **maxPlace** - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
- **numGroups** - Number of groups we have data for in the match.
- **revives** - Number of times this player revived teammates.
- **rideDistance** - Total distance traveled in vehicles measured in meters.
- **roadKills** - Number of kills while in a vehicle.
- **swimDistance** - Total distance traveled by swimming measured in meters.
- **teamKills** - Number of times this player killed a teammate.
- **vehicleDestroys** - Number of vehicles destroyed.
- **walkDistance** - Total distance traveled on foot measured in meters.
- **weaponsAcquired** - Number of weapons picked up.
- **winPoints** - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)
- **winPlacePerc** - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

## Считывание и подготовка данных

In [0]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_csv('train_V2.csv.zip', compression='zip')

In [0]:
orig_cols = train.columns

Пустые поля в датасете очень мешают методам машинного обучения работать с данными. Можно с ними бороться так:

In [0]:
train.dropna(inplace = True)

In [0]:
def original(df):
    return df

In [0]:
def total_distance(df):
    df['total_distance'] = df['rideDistance'] + df['swimDistance'] + df['walkDistance']
    return df

In [0]:
def players_in_team(df):
    df['players_in_team'] = df.groupby('groupId')['groupId'].transform('count')
    return df

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def run_experiment(preprocess):
   
    train_exp = train[orig_cols]
    df_exp = preprocess(train_exp)

    target = 'winPlacePerc'
    cols_to_drop = ['Id', 'groupId', 'matchId', 'matchType', target]
    cols_to_fit = [col for col in df_exp.columns if col not in cols_to_drop]
    
    X_train, X_test, y_train, y_test = train_test_split(train_exp.drop('winPlacePerc', axis=1),
                                                    train_exp['winPlacePerc'],
                                                    test_size=0.2,
                                                    random_state=42)
    
    model = LinearRegression()
    model.fit(X_train[cols_to_fit], y_train)
    
    y_pred = model.predict(X_test[cols_to_fit])
    return mean_absolute_error(y_test, y_pred)

def run_experiments(preprocesses):
    results = []
    for preprocess in preprocesses:
        start = time.time()
        score = run_experiment(preprocess)
        execution_time = time.time() - start
        results.append({
            'name': preprocess.__name__,
            'score': score,
            'execution time': f'{round(execution_time, 2)}s'
        })
        
    return pd.DataFrame(results, columns=['name', 'score', 'execution time']).sort_values(by='score')

In [9]:
run_experiments([
    original,players_in_team, total_distance])

,name,score,execution time
1,players_in_team,0.091600,23.32s
0,original,0.092592,15.83s
2,total_distance,0.092593,14.83s


In [9]:
run_experiment(players_in_team)

0.0916003481692193

In [11]:
run_experiment(total_distance)

0.09259335276874997